In [1]:
import sys
sys.path.insert(0,'..')

%load_ext autoreload
%autoreload 2

import unittest
import numpy as np

from bayes.parameters import ParameterList
from bayes.inference_problem import VariationalBayesProblem, ModelErrorInterface
from bayes.noise import UncorrelatedNoiseTerm

from bayes.noise import UncorrelatedNoiseTerm

In [2]:
class Sensor:
    def __init__(self, name, shape=(1, 1)):
        self.name = name
        self.shape = shape

    def __repr__(self):
        return f"{self.name} {self.shape}"


class MySensor(Sensor):
    def __init__(self, name, position):
        super().__init__(name, shape=(1, 1))
        self.position = position


class MyForwardModel:
    def __call__(self, parameter_list, sensors, time_steps):
        """
        evaluates 
            fw(x, t) = A * x + B * t
        """
        A = parameter_list["A"]
        B = parameter_list["B"]

        result = {}
        for sensor in sensors:
            result[sensor] = A * sensor.position + B * time_steps
        return result

    def parameter_list(self):
        p = ParameterList()
        p.define("A", None)
        p.define("B", None)
        return p


class MyModelError(ModelErrorInterface):
    def __init__(self, fw, data):
        self._fw = fw
        self._ts, self._sensor_data = data
        self.parameter_list = fw.parameter_list()

    def __call__(self):
        sensors = list(self._sensor_data.keys())
        model_response = self._fw(self.parameter_list, sensors, self._ts)
        error = {}
        for sensor in sensors:
            error[sensor] = model_response[sensor] - self._sensor_data[sensor]
        return error

In [41]:
class PytorchProblem(VariationalBayesProblem):
    def __init__(self):
        super().__init__()

    def posterior_pytorch_model(self):
        #sampled_parameters = ParamStoreDict()
        sampled_parameters = {}

        for name, (mean, sd) in self.prm_prior.items():
            # idx = problem.latent[name].start_idx
            assert self.latent[name].N == 1  # vector parameters not yet supported!
            sampled_parameters[name] = pyro.sample(name, dist.Normal(mean, sd))

        for name, gamma in self.noise_prior.items():
            # idx = problem.latent[name].start_idx
            shape, scale = gamma.shape, gamma.scale
            alpha, beta = shape, 1.0 / scale
            # check for Gamma and Inverse Gamma
            sampled_noise_precision = pyro.sample(name, dist.Gamma(alpha, beta))
            self.noise_models[name].parameter_list['precision'] = sampled_noise_precision

        for noise_key, noise_term in self.noise_models.items():
            pyro.sample(noise_key + "_latent",
                        dist.Normal(self.wrapper_function(sampled_parameters, noise_key),
                        noise_term.parameter_list['precision']),
                        obs=torch.zeros(len(vector)))
        return

    def wrapper_function(self, sampled_parameters, noise_key):
        for name, value in sampled_parameters.items():
             #self.latent[name].set_value(value)
            self.latent[name].value(value)

        #self.latent['test'].value(sampled_parameters)
        # compute raw model error using the samples parameters, this is now done for each noise term separately
        raw_me = {}
        for key, me in self.model_errors.items():
            raw_me[key] = me()

        vector = self.noise_models[noise_key].vector_contribution(raw_me)
        return torch.from_numpy(vector)

In [27]:
temp ={}
for a, (b,c) in problem.prm_prior.items():
    temp[a] = pyro.sample(a,dist.Normal(b,c))

In [43]:
forw = problem.wrapper_function(temp,'noise0')

IndexError: invalid index of a 0-dim tensor. Use `tensor.item()` in Python or `tensor.item<T>()` in C++ to convert a 0-dim tensor to a number

In [42]:
s1, s2, s3 = MySensor("S1", 0.2), MySensor("S2", 0.5), MySensor("S3", 42.0)

fw = MyForwardModel()
prm = fw.parameter_list()

# set the correct values
A_correct = 42.0
B_correct = 6174.0
prm["A"] = A_correct
prm["B"] = B_correct

np.random.seed(6174)
noise_sd1 = 0.2
noise_sd2 = 0.4

def generate_data(N_time_steps, noise_sd):
    time_steps = np.linspace(0, 1, N_time_steps)
    model_response = fw(prm, [s1, s2, s3], time_steps)
    sensor_data = {}
    for sensor, perfect_data in model_response.items():
        sensor_data[sensor] = perfect_data + np.random.normal(
            0.0, noise_sd, N_time_steps
        )
    return time_steps, sensor_data

data1 = generate_data(101, noise_sd1)
data2 = generate_data(51, noise_sd2)

me1 = MyModelError(fw, data1)
me2 = MyModelError(fw, data2)

problem = PytorchProblem()
key1 = problem.add_model_error(me1)
key2 = problem.add_model_error(me2)

problem.latent["A"].add(me1.parameter_list, "A")
problem.latent["A"].add(me2.parameter_list, "A")
# or simply
problem.define_shared_latent_parameter_by_name("B")

problem.set_normal_prior("A", 40.0, 5.0)
problem.set_normal_prior("B", 6000.0, 300.0)

noise1 = UncorrelatedNoiseTerm()
noise1.add(s1, key1)
noise1.add(s2, key1)
noise1.add(s3, key1)

noise2 = UncorrelatedNoiseTerm()
noise2.add(s1, key2)
noise2.add(s2, key2)
noise2.add(s3, key2)

noise_key1 = problem.add_noise_model(noise1)
noise_key2 = problem.add_noise_model(noise2)

problem.set_noise_prior(noise_key1, 3 * noise_sd1, sd_shape=0.5)
problem.set_noise_prior(noise_key2, 3 * noise_sd2, sd_shape=0.5)

In [17]:
chk = {}
chk['a']=1
chk['b']=2
chk

{'a': 1, 'b': 2}

In [6]:
import arviz as az
import torch
import pyro
import pyro.distributions as dist
import pyro.poutine as poutine
from pyro.infer import MCMC, NUTS, Predictive
nuts_kernel = NUTS(problem.posterior_pytorch_model, jit_compile=True)
mcmc = MCMC(nuts_kernel,
            num_samples=300,
            warmup_steps=50,
            num_chains=4
            )
mcmc.run()
mcmc.summary(prob=0.5)
posterior_samples = mcmc.get_samples()
posterior_predictive = Predictive(problem.posterior_pytorch_model, posterior_samples)()
prior = Predictive(problem.posterior_pytorch_model, num_samples=500)()
pm_data = az.from_pyro(
    mcmc,
    prior=prior,
)
fig, axes = plt.subplots(1, 3, figsize=(11, 5), squeeze=False)
az.plot_dist_comparison(pm_data, var_names=["E", "noise0"], figsize=(8, 9))
az.plot_trace(pm_data, figsize=(11, 4))
az.plot_pair(pm_data, kind="scatter", ax=axes[0, 0])
# az.plot_posterior(pm_data, kind="hist")
az.plot_posterior(pm_data, var_names="E", kind="hist", ax=axes[0, 1])
az.plot_posterior(pm_data, var_names="noise0", kind="hist", ax=axes[0, 2])
plt.show()

[ERROR LOG CHAIN:0]invalid index of a 0-dim tensor. Use `tensor.item()` in Python or `tensor.item<T>()` in C++ to convert a 0-dim tensor to a number
Traceback (most recent call last):
  File "/home/atul_0711/miniconda3/envs/pytorch/lib/python3.8/site-packages/pyro/infer/mcmc/api.py", line 101, in run
    for sample in _gen_samples(self.kernel, self.warmup_steps, self.num_samples, logging_hook,
  File "/home/atul_0711/miniconda3/envs/pytorch/lib/python3.8/site-packages/pyro/infer/mcmc/api.py", line 113, in _gen_samples
    kernel.setup(warmup_steps, *args, **kwargs)
  File "/home/atul_0711/miniconda3/envs/pytorch/lib/python3.8/site-packages/pyro/infer/mcmc/hmc.py", line 302, in setup
    self._initialize_model_properties(args, kwargs)
  File "/home/atul_0711/miniconda3/envs/pytorch/lib/python3.8/site-packages/pyro/infer/mcmc/hmc.py", line 244, in _initialize_model_properties
    init_params, potential_fn, transforms, trace = initialize_model(
  File "/home/atul_0711/miniconda3/envs/pyto

[ERROR LOG CHAIN:2]invalid index of a 0-dim tensor. Use `tensor.item()` in Python or `tensor.item<T>()` in C++ to convert a 0-dim tensor to a number
Traceback (most recent call last):
  File "/home/atul_0711/miniconda3/envs/pytorch/lib/python3.8/site-packages/pyro/infer/mcmc/api.py", line 101, in run
    for sample in _gen_samples(self.kernel, self.warmup_steps, self.num_samples, logging_hook,
  File "/home/atul_0711/miniconda3/envs/pytorch/lib/python3.8/site-packages/pyro/infer/mcmc/api.py", line 113, in _gen_samples
    kernel.setup(warmup_steps, *args, **kwargs)
  File "/home/atul_0711/miniconda3/envs/pytorch/lib/python3.8/site-packages/pyro/infer/mcmc/hmc.py", line 302, in setup
    self._initialize_model_properties(args, kwargs)
  File "/home/atul_0711/miniconda3/envs/pytorch/lib/python3.8/site-packages/pyro/infer/mcmc/hmc.py", line 244, in _initialize_model_properties
    init_params, potential_fn, transforms, trace = initialize_model(
  File "/home/atul_0711/miniconda3/envs/pyto

IndexError: invalid index of a 0-dim tensor. Use `tensor.item()` in Python or `tensor.item<T>()` in C++ to convert a 0-dim tensor to a number